In [1]:
import time
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd

## To Do
### 1. Scrape Cash from Cash urls

In [2]:
#Order -> Active, Reserve/Susp, IR, Practice Squad, Dead, Summary

In [3]:
Teams = ["arizona-cardinals", "atlanta-falcons", 'baltimore-ravens', "buffalo-bills", "carolina-panthers", "chicago-bears",
        "cincinnati-bengals", "cleveland-browns", "dallas-cowboys", "denver-broncos", "detroit-lions", "green-bay-packers",
        "houston-texans", "indianapolis-colts", "jacksonville-jaguars", "kansas-city-chiefs", "oakland-raiders", "san-diego-chargers",
        "los-angeles-rams", "miami-dolphins", "minnesota-vikings", "new-england-patriots", "new-orleans-saints", "new-york-giants",
        "new-york-jets", "philadelphia-eagles", "pittsburgh-steelers", "san-francisco-49ers", "seattle-seahawks", "tampa-bay-buccaneers",
        "tennessee-titans", "washington-redskins"]
Years = range(2005,2020)

In [4]:
##Define program that turns URL into a list of tables that the URL contains
def Soupify(WebAddress):
    url = WebAddress
    cookies = {
        'PHPSESSID': '1roujrc16bd9p5ojugiv1pfablfb9qmo',
        '_ga': 'GA1.2.850172306.1592854911',
        '_gid': 'GA1.2.1355902833.1592854911',
        '_gat': '1',
    }

    headers = {
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-User': '?1',
        'Sec-Fetch-Dest': 'document',
        'Referer': 'https://www.spotrac.com/signin/',
        'Accept-Language': 'en-US,en;q=0.9',
    }
    response = requests.get(url, headers=headers, cookies=cookies)
    html = response.content
    soup = BeautifulSoup(html)
    return soup
def GetNum(Soup):
    alltext = Soup.get_text()
    IR = 0
    if "Reserve/Suspended" in alltext:
        RS = 1
    else:
        RS = 0
    if "Practice Squad" in alltext:
        PS = 1
    else:
        PS = 0
        
    tables = Soup.findAll('table')
    
    tab = tables[-1]
    rows = tab.find_all('td')
    for row in rows:
        txt = row.get_text()
        if "Injured Reserve" in txt:
            IR = 1
    
    print (RS,PS,IR)
    return RS,PS,IR
    
def Tablify(Soup):
    tables = Soup.findAll('table')
    return tables

In [5]:
def Scraping(Teams,Years):
    ActiveData = pd.DataFrame() 
    for team in Teams:
        for year in Years:
            Active = []
            ResSusp = []
            IR = []
            PracSquad = []
            Dead = []
            Summary = []
            url = "https://www.spotrac.com/nfl/" + team + "/cap/" + str(year) + "/"
            soup = Soupify(url)
            RS, PS, IR = GetNum(soup)
            tables = Tablify(soup)
            print (team)
            print (year)
            if RS == 1:
                if PS == 1:
                    if IR == 1:
                        Active = tables[0]
                        ResSusp = tables[1]
                        IR = tables[2]
                        PracSquad = tables[3]
                        Dead = tables[4]
                        Summary = tables[5]
                    elif IR == 0:
                        Active = tables[0]
                        ResSusp = tables[1]
                        PracSquad = tables[2]
                        Dead = tables[3]
                        Summary = tables[4]
                elif PS == 0:
                    if IR == 1:
                        Active = tables[0]
                        ResSusp = tables[1]
                        IR = tables[2]
                        Dead = tables[3]
                        Summary = tables[4]
                    elif IR == 0:
                        Active = tables[0]
                        ResSusp = tables[1]
                        Dead = tables[2]
                        Summary = tables[3]
            elif RS == 0:
                if PS == 1:
                    if IR == 1:
                        Active = tables[0]
                        IR = tables[1]
                        PracSquad = tables[2]
                        Dead = tables[3]
                        Summary = tables[4]
                    elif IR == 0:
                        Active = tables[0]
                        PracSquad = tables[1]
                        Dead = tables[2]
                        Summary = tables[3]
                elif PS == 0:
                    if IR == 1:
                        Active = tables[0]
                        IR = tables[1]
                        Dead = tables[2]
                        Summary = tables[3]
                    elif IR == 0:
                        Active = tables[0]
                        Dead = tables[1]
                        Summary = tables[2]

            ActivePlayerData = ActiveScrape(Active, team, year)
            ActiveData = ActiveData.append(ActivePlayerData)

            if str(type(ResSusp)) == "<class 'bs4.element.Tag'>":
                ResSuspData = ResScrape(ResSusp,team,year)
                ActiveData = ActiveData.append(ResSuspData)
            if str(type(IR)) == "<class 'bs4.element.Tag'>":
                IRData = IRScrape(IR,team,year)
                ActiveData = ActiveData.append(IRData)
            if str(type(PracSquad)) == "<class 'bs4.element.Tag'>":
                PracData = PSquadScrape(PracSquad, team, year)
                ActiveData = ActiveData.append(PracData)
            if str(type(Dead)) == "<class 'bs4.element.Tag'>":
                DeadData = DeadScrape(Dead, team, year)
                ActiveData = ActiveData.append(DeadData)
                
    return ActiveData

In [6]:
def ActiveScrape (Active, team, year):
    players = Active.find_all('tr')
    players.pop(0)
    players.pop(-1)
    OneTeamOneYear = pd.DataFrame()
    for i in range(0,len(players)):

        cells = players[i].find_all('td')

        ##Name
        name = cells[0].get_text().rstrip().split('\n')[1]
        ##Position
        position = cells[1].get_text()
        ##Cash
        cash = cells[2].get_text()
        ##Cap Hit
        salary = cells[10].get_text().lstrip("$").replace(",","")
        #salary = float(salary)
        ##Cap%
        capperc = cells[11].get_text()
        capperc = float(capperc)
        
        
        IR = 0
        Suspended = 0
        PracSquad = 0
        DeadCap = 0
        list_of_attributes = {"Team": team ,"Year": year,"Player": name,"Position": position,"SalaryCap": salary, "Cash": cash,
                              "CapPerc": capperc,"IR": IR,"Susp": Suspended, "Prac": PracSquad, "DeadCap": DeadCap}
        OneTeamOneYear = OneTeamOneYear.append(list_of_attributes, ignore_index = True)
        
    return OneTeamOneYear

def ResScrape (Active, team, year):
    players = Active.find_all('tr')
    players.pop(0)
    players.pop(-1)
    OneTeamOneYear = pd.DataFrame()
    for i in range(0,len(players)):

        cells = players[i].find_all('td')

        ##Name
        name = cells[0].get_text().rstrip().split('\n')[1]
        ##Position
        position = cells[1].get_text()
        ##Cash
        cash = cells[2].get_text()
        ##Cap Hit
        salary = cells[10].get_text().lstrip("$").replace(",","")
        #salary = float(salary)
        ##Cap%
        capperc = cells[11].get_text()
        capperc = float(capperc)
        
        
        IR = 0
        Suspended = 1
        PracSquad = 0
        DeadCap = 0
        list_of_attributes = {"Team": team ,"Year": year,"Player": name,"Position": position,"SalaryCap": salary, "Cash": cash,
                              "CapPerc": capperc,"IR": IR,"Susp": Suspended, "Prac": PracSquad, "DeadCap": DeadCap}
        OneTeamOneYear = OneTeamOneYear.append(list_of_attributes, ignore_index = True)
        
    return OneTeamOneYear

def IRScrape (Active, team, year):
    players = Active.find_all('tr')
    players.pop(0)
    players.pop(-1)
    OneTeamOneYear = pd.DataFrame()
    for i in range(0,len(players)):

        cells = players[i].find_all('td')

        ##Name
        name = cells[0].get_text().rstrip().split('\n')[1]
        ##Position
        position = cells[1].get_text()
        ##Cash
        cash = cells[2].get_text()
        ##Cap Hit
        salary = cells[10].get_text().lstrip("$").replace(",","")
        #salary = float(salary)
        ##Cap%
        capperc = cells[11].get_text()
        capperc = float(capperc)
        
        
        IR = 1
        Suspended = 0
        PracSquad = 0
        DeadCap = 0
        list_of_attributes = {"Team": team ,"Year": year,"Player": name,"Position": position,"SalaryCap": salary, "Cash": cash,
                              "CapPerc": capperc,"IR": IR,"Susp": Suspended, "Prac": PracSquad, "DeadCap": DeadCap}
        
        OneTeamOneYear = OneTeamOneYear.append(list_of_attributes, ignore_index = True)
        
    return OneTeamOneYear

def PSquadScrape (Active, team, year):
    players = Active.find_all('tr')
    players.pop(0)
    players.pop(-1)
    OneTeamOneYear = pd.DataFrame()
    for i in range(0,len(players)):

        cells = players[i].find_all('td')

        ##Name
        name = cells[0].get_text().rstrip().split('\n')[1]
        ##Position
        position = cells[1].get_text()
        ##Cash
        cash = cells[2].get_text()
        ##Cap Hit
        salary = cells[10].get_text().lstrip("$").replace(",","")
        #salary = float(salary)
        ##Cap%
        capperc = cells[11].get_text()
        capperc = float(capperc)
        
        
        IR = 0
        Suspended = 0
        PracSquad = 1
        DeadCap = 0
        list_of_attributes = {"Team": team ,"Year": year,"Player": name,"Position": position,"SalaryCap": salary, "Cash": cash,
                              "CapPerc": capperc,"IR": IR,"Susp": Suspended, "Prac": PracSquad, "DeadCap": DeadCap}
        OneTeamOneYear = OneTeamOneYear.append(list_of_attributes, ignore_index = True)
        
    return OneTeamOneYear

def DeadScrape (Active, team, year):
    players = Active.find_all('tr')
    players.pop(0)
    players.pop(-1)
    OneTeamOneYear = pd.DataFrame()
    for i in range(0,len(players)):

        cells = players[i].find_all('td')

        ##Name
        name = cells[0].get_text().rstrip().split('\n')[1]
        ##Position
        position = cells[1].get_text()
        ##Cash Hit
        cash = cells[2].get_text()
        ##Cap Hit
        salary = cells[10].get_text().lstrip("$").replace(",","")
        #salary = float(salary)
        ##Cap%
        capperc = cells[11].get_text()
        capperc = float(capperc)
        
        
        IR = 0
        Suspended = 0
        PracSquad = 0
        DeadCap = 1
        list_of_attributes = {"Team": team ,"Year": year,"Player": name,"Position": position,"SalaryCap": salary, "Cash": cash,
                              "CapPerc": capperc,"IR": IR,"Susp": Suspended, "Prac": PracSquad, "DeadCap": DeadCap}
        OneTeamOneYear = OneTeamOneYear.append(list_of_attributes, ignore_index = True)
        
    return OneTeamOneYear


In [7]:
df = Scraping(Teams, Years)

0 1 1
arizona-cardinals
2005
0 1 1
arizona-cardinals
2006
0 1 1
arizona-cardinals
2007
0 1 1
arizona-cardinals
2008
0 1 1
arizona-cardinals
2009
0 1 1
arizona-cardinals
2010
0 1 1
arizona-cardinals
2011
0 0 1
arizona-cardinals
2012
0 1 1
arizona-cardinals
2013
1 1 1
arizona-cardinals
2014
1 0 1
arizona-cardinals
2015
1 1 1
arizona-cardinals
2016
0 1 1
arizona-cardinals
2017
0 0 1
arizona-cardinals
2018
0 1 1
arizona-cardinals
2019
0 1 0
atlanta-falcons
2005
0 0 1
atlanta-falcons
2006
1 1 1
atlanta-falcons
2007
1 1 1
atlanta-falcons
2008
0 1 1
atlanta-falcons
2009
0 1 0
atlanta-falcons
2010
0 1 1
atlanta-falcons
2011
0 1 1
atlanta-falcons
2012
0 1 1
atlanta-falcons
2013
0 1 1
atlanta-falcons
2014
0 0 1
atlanta-falcons
2015
0 1 1
atlanta-falcons
2016
0 1 1
atlanta-falcons
2017
0 0 1
atlanta-falcons
2018
1 1 1
atlanta-falcons
2019
0 1 1
baltimore-ravens
2005
0 0 1
baltimore-ravens
2006
0 1 1
baltimore-ravens
2007
0 1 1
baltimore-ravens
2008
0 1 1
baltimore-ravens
2009
0 1 1
baltimore-rave

1 0 1
miami-dolphins
2015
0 0 1
miami-dolphins
2016
0 0 1
miami-dolphins
2017
0 0 1
miami-dolphins
2018
0 1 1
miami-dolphins
2019
0 0 0
minnesota-vikings
2005
0 1 1
minnesota-vikings
2006
0 0 1
minnesota-vikings
2007
0 1 1
minnesota-vikings
2008
0 1 1
minnesota-vikings
2009
0 1 0
minnesota-vikings
2010
0 1 1
minnesota-vikings
2011
0 1 1
minnesota-vikings
2012
0 1 1
minnesota-vikings
2013
0 1 1
minnesota-vikings
2014
0 0 1
minnesota-vikings
2015
0 0 1
minnesota-vikings
2016
0 0 1
minnesota-vikings
2017
0 0 1
minnesota-vikings
2018
0 1 1
minnesota-vikings
2019
0 0 1
new-england-patriots
2005
0 0 0
new-england-patriots
2006
0 1 0
new-england-patriots
2007
0 1 1
new-england-patriots
2008
0 1 0
new-england-patriots
2009
0 1 1
new-england-patriots
2010
0 1 1
new-england-patriots
2011
0 1 1
new-england-patriots
2012
0 1 1
new-england-patriots
2013
0 1 1
new-england-patriots
2014
0 1 1
new-england-patriots
2015
0 0 1
new-england-patriots
2016
0 1 1
new-england-patriots
2017
1 0 1
new-england-p

KeyboardInterrupt: 

In [ ]:
if str(type(IR)) == "<class 'bs4.element.Tag'>":
    print (1)
else:
    print (0)

In [ ]:
str(type(Active))

In [ ]:
df.to_csv("PlayerSalaryData.csv", index = False)

In [ ]:
type(Active)

### Scrape teams W-L and playoffs

In [42]:
teams = ['bears','bengals','bills','broncos','browns','buccaneers','cardinals','chargers','chiefs','colts','cowboys',
         'dolphins','eagles','falcons','49ers','giants','jaguars','jets','lions','packers','panthers','patriots','ravens',
        'rams','raiders','redskins','saints','seahawks','steelers','texans','titans','vikings']
Years = range(2005,2020)

In [50]:
def Soupify(WebAddress):
    url = WebAddress
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html)
    tables = soup.findAll('table')
    return tables


def ReadTables(Tables):
    TeamTable = pd.DataFrame(columns = ['Team','Year','Wins','Postseason'])
    table = Tables[2]
    for row in table.findAll('tr'):
        cells = row.findAll('td')
        if len(cells) > 0:           
            Year = cells[0].text
            Record = cells[1].text.split("-")
            Wins = Record[0]
            Postseason = cells[3]
            TeamRow = {'Year': Year, 'Wins':Wins, 'Postseason':Postseason}
            TeamTable = TeamTable.append(TeamRow, ignore_index = True)
    return TeamTable


            
                
    
    
Teamsly = pd.DataFrame(columns = ['Team','Year','Wins','Postseason'])   
for thisteam in teams:
    URL = "http://www.jt-sw.com/football/pro/teams.nsf/histories/" + thisteam
    print (URL)
    Tables = Soupify(URL)
    Team = ReadTables(Tables)
    Team['Team'] = thisteam
    Teamsly = Teamsly.append(Team,ignore_index = True)


http://www.jt-sw.com/football/pro/teams.nsf/histories/bears
http://www.jt-sw.com/football/pro/teams.nsf/histories/bengals
http://www.jt-sw.com/football/pro/teams.nsf/histories/bills
http://www.jt-sw.com/football/pro/teams.nsf/histories/broncos
http://www.jt-sw.com/football/pro/teams.nsf/histories/browns
http://www.jt-sw.com/football/pro/teams.nsf/histories/buccaneers
http://www.jt-sw.com/football/pro/teams.nsf/histories/cardinals
http://www.jt-sw.com/football/pro/teams.nsf/histories/chargers
http://www.jt-sw.com/football/pro/teams.nsf/histories/chiefs
http://www.jt-sw.com/football/pro/teams.nsf/histories/colts
http://www.jt-sw.com/football/pro/teams.nsf/histories/cowboys
http://www.jt-sw.com/football/pro/teams.nsf/histories/dolphins
http://www.jt-sw.com/football/pro/teams.nsf/histories/eagles
http://www.jt-sw.com/football/pro/teams.nsf/histories/falcons
http://www.jt-sw.com/football/pro/teams.nsf/histories/49ers
http://www.jt-sw.com/football/pro/teams.nsf/histories/giants
http://www.jt

In [62]:
Teamsly['Year'] = Teamsly['Year'].astype(int)
Teamsly = Teamsly[Teamsly['Year'] > 2004]

In [64]:
Teamsly

,Team,Year,Wins,Postseason
0,bears,2019,8,[--]
1,bears,2018,12,[0-1 - Lost Wild Card Playoffs]
2,bears,2017,5,[--]
3,bears,2016,3,[--]
4,bears,2015,6,[--]
...,...,...,...,...
1975,vikings,2009,12,[1-1 - Lost Conference Championship]
1976,vikings,2008,10,[0-1 - Lost Wild Card Playoffs]
1977,vikings,2007,8,[--]
1978,vikings,2006,6,[--]


In [93]:
def Playoffs(x):
    POText = x['Postseason'].text
    Wins = 0
    if len(POText) > 2:
        bits = POText.split('-')
        Wins = int(bits[0])
        Losses = int(bits[1])
        Text = bits[2]
        if (Wins + Losses == 1 ) & ('Divisional Playoffs' in Text):
            Wins += 1
        elif (Wins + Losses == 2) & ('Conference Championship' in Text):
            Wins += 1
        elif (Wins + Losses == 3) & ('Superbowl' in Text):
            Wins += 1
            
        
    return Wins

def SuperBowl(x):
    POText = x['Postseason'].text
    SBWin = 0
    if len(POText) > 2:
        bits = POText.split('-')
        Text = bits[2]
        if ("Won Superbowl") in Text:
            SBWin = 1
    return SBWin
        

Teamsly['Playoff Wins'] = Teamsly.apply(Playoffs, axis = 1)
Teamsly['SuperBowl Win'] = Teamsly.apply(SuperBowl, axis = 1)

In [96]:
Teamsly = Teamsly.drop(['Postseason','POText'], axis = 1)

In [97]:
Teamsly.to_csv("TeamsWins.csv", index = False)